# Практическое занятие 01. Итерация и рекурсия 

**Итерация** и **рекурсия** - это два подхода к составлению алгоритмов, использующих повторение. Они взаимозаменяемы, но обычно используются в разных задачах. Итерация, как правило, более экономна к расходу памяти и достаточно быстродейственна, а рекурсия мо- жет приводить к более лаконичным и красивым решениям.


### Задача 1. Вычисление факториала целого числа

Запишем первый способ вычисления факториала:

$ n! = n * (n-1) * (n-2) * ... * 1 $

Эта запись предполагает итеративное решение (с помощью цикла)

In [7]:
#include <iostream>

In [2]:
int fact1(int n) {
    int result=1;
    for(int i=n; i>=1; i--)
       result *=i;
    return result;
}
std::cout << fact1(5); // 5*4*3*2*1

120

@0x10974bc30

Есть другой способ итеративного вычисления факториала:

$ n! = 1 * 2 * .... * (n-1) * n$

Соответствующая программа может выглядеть так:

In [2]:
int fact2(int n) {
    int result=1;
    for(int i=1; i<=n; i++)
       result *=i;
    return result;
}
std::cout << fact2(5); 

120

Теперь запишем вычисление факториала в рекурсивном виде

$ n! = n * (n-1)!, n>1 $

$ n! = 1,          n=1 $

In [3]:
int fact3(int n) {
    if(n==1)
        return 1;
    else
        return n * fact3(n-1);
}
std::cout << fact3(5); 

120

В качестве достоинства приведенного решения можно указать отсутствие вспомогатель- ных (временных) переменных. На самом деле, промежуточные результаты сохраняются в стековой памяти.

![](./fact.png)

### Задача 2. Вычисление суммы элементов массива

Итеративный способ

Запишем выражение суммы для массива, в котором $S_i$ - сумма в массиве из одного элемента на позиции i

$ S = S_1 + S_2 + ... + S_n $

Нахождение суммы элемента массива итеративным способом предполагает использование аккумулятора (sum), начальное значение которого обнуляется. В процессе работы алгоритма мы обращаемся последовательно к каждому элементу массива по смещению (i) и приплюсовываем его значение к аккумулятору.


In [3]:
int arrSum1(int a[], int n) {
    int sum=0;
    for(int i=0;i<n;i++)
        sum+=a[i];
    return sum;
}

const int n = 5;
int arr[n]={6,7,8,9,0};
std::cout << arrSum1(arr,n);

30

Рекурсивный способ

Массив разбивается на два подмассива, поэтому сумма считается как сумма сумм подмассивов

$ S = S_1 + S_2..n, n>1 $

$ S = S_1, n=1 $

In [15]:
int arrSum2(int a[], int n) {
    if(n==1)
        return a[0];
    else
        return a[0]+arrSum2(a+1,n-1);
}
const int n = 5;
int arr[n]={6,7,8,9,0};
std::cout << arrSum2(arr,n);

30

Недостатком приведенной реализации является большая глубина рекурсии, почти равная размеру массива. Для крупных массивов это может привести к переполнению стека и аварийному завершению программы.

Вторая реализация использует разбиение массива на две равные (почти равные) части и суммирование сумм этих частей. Благодаря такому разделению, глубина рекурсии будет пропорциональна $log(n)$, что безопасно с точки зрения переполнения стека.

In [ ]:
      1 2 + 3 4 5
    1 + 2 | 3 + 4 5
          |     4 + 5    

In [8]:
int arrSum3(int a[], int n) {
    if(n==1)
        return a[0];
    else
        return arrSum3(a,n/2)+arrSum3(a+n/2,n-n/2);
}
const int n = 5;
int arr[n]={6,7,8,9,0};
std::cout << arrSum3(arr,n);

30

### Задача 3. Вычисление ряда Фибоначчи

Рассмотрим задачу нахождения n-ого члена ряда Фибоначчи
0,1,1,2,3,5,8,13,21,34,55...
В этом ряду определяются элементы с номерами 1 и 2, а последующие элементы считаются как сумма двух предыдущих элементов

In [13]:
#include <iostream>
#include <ctime>
#include <cstdint>

In [16]:
// 0,1,1,2,3,5,8,13,21,34,55...

uint64_t fib1(uint64_t n) 
{
    if(n==1)
        return 0;
    if(n==2)
        return 1;
    else
        return fib1(n-1) + fib1(n-2);
}

Для исследования времени работы функции fib создадим цикл, в котором будем счиать время работы для n=30,32,34,...,44
После измерений построим график зависимости времени работы от n.

In [17]:
for(int i = 0, n = 30; i < 7; i++, n+=1) {
   clock_t start = clock();
   uint64_t result = fib1(n);
   clock_t finish = clock();
   std::cout << (double)(finish-start)/CLOCKS_PER_SEC << std::endl;
}

0.003833
0.007286
0.01098
0.017483
0.027007
0.040678
0.065451


Нетрудно видеть, что время растет по экспоненциальному закону. Это вызвано тем, что при вычислениях мы не где не сохраняем ранее вычислнные значения и их приходится считать по 2 раза (для n-1 и для n-2). В результате разрастается дерево рекурсии.

![](./fib-tree.png)

Теперь изменим решение, перейдя от экспоненциальной рекурсии к линейной. Для этого, накопленная сумма двух элементов передается при рекурсивном вызове. Мы также передаем счетчик, который считает от n до 0 и 0 выступает условием выхода из рекурсии.

In [4]:
// 0,1,1,2,3,5,8,13,21,34,...
uint64_t fibIter(uint64_t a, uint64_t b, uint64_t count) {
    if(count==0)
        return b;
    else
        return fibIter(a + b, a, --count);
}

In [5]:
// decorate
uint64_t fib2(uint64_t n) {
    return fibIter(0,1,n);
}

In [8]:
std::cout << fib2(50) << std::endl;

7778742049
